In [2]:
import requests
import os
from pathlib import Path
import tqdm


import asyncio
import aiohttp
import aiofiles 
from tqdm.asyncio import tqdm_asyncio

In [3]:
class SPBU_scrapper():
    WORKS_LIMIT = 2000
        
    def check_downloaded(self):
        self.works_to_download = [f"work_{i}.pdf" for i in range(1, self.WORKS_LIMIT)]        
        
        for path in os.listdir(f"{self.data_dir}/spbu/pdf"):
            if os.path.isfile(os.path.join(f"{self.data_dir}/spbu/pdf", path)):
                self.works_to_download.remove(path)

    def __init__(self, data_dir="data"):
        self.data_dir = data_dir
        path = Path(f"{data_dir}/spbu/pdf").mkdir(parents=True, exist_ok=True)
        
        self.check_downloaded()
        
        print(f"Found {self.WORKS_LIMIT - len(self.works_to_download)} pdfs")
        print()
    
    async def parse(self):

        error_works = []
        
        async def fetch_file(work):
            work_id = work[5:-4]
            url = f"https://se.math.spbu.ru/thesis_download?thesis_id={work_id}"
            
            async with aiohttp.ClientSession() as session:
                async with session.get(url) as resp:
                    if resp.status == 200:
                        data = await resp.read()
                        async with aiofiles.open(
                            os.path.join(self.data_dir, 'spbu/pdf', work), "wb"
                        ) as outfile:
                            await outfile.write(data)
                        
        tasks = [fetch_file(work) for work in self.works_to_download]
        await tqdm_asyncio.gather(*tasks)
        self.check_downloaded()

In [5]:
scrapper = SPBU_scrapper()
await scrapper.parse()

Found 1094 pdfs



100%|████████████████████████████████████████████████████████████████████████████████| 906/906 [00:10<00:00, 85.12it/s]


## TODO
- составить список пдф парсеров
- протестить весь список
- подумать как можно обьединять результаты(сделать из двух какашек золото)